In [1]:
import folium
import pandas as pd
import requests
import io
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [2]:

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_df = pd.read_csv(io.StringIO(response.text))
spacex_df.tail(10)

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
46,43,2017-10-11,22:53:00,F9 FT B1031.2,KSC LC-39A,SES-11 / EchoStar 105,5200.00,GTO,SES EchoStar,Success (drone ship),1,28.573255,-80.646895
47,44,2017-10-30,19:34:00,F9 B4 B1042.1,KSC LC-39A,Koreasat 5A,3500.00,GTO,KT Corporation,Success (drone ship),1,28.573255,-80.646895
48,54,2018-05-11,20:14:00,F9 B5 B1046.1,KSC LC-39A,Bangabandhu-1,3600.00,GTO,Thales-Alenia/BTRC,Success (drone ship),1,28.573255,-80.646895
49,45,2017-12-15,15:36:00,F9 FT B1035.2,CCAFS SLC-40,SpaceX CRS-13,2205.00,LEO (ISS),NASA (CRS),Success (ground pad),1,28.563197,-80.576820
50,47,2018-01-08,1:00:00,F9 B4 B1043.1,CCAFS SLC-40,Zuma,3696.65,LEO,Northrop Grumman,Success (ground pad),1,28.563197,-80.576820
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.00,GTO,SES,Controlled (ocean),0,28.563197,-80.576820
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.00,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.576820
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.00,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.576820
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.00,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.576820
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.00,GTO,SES,No attempt,0,28.563197,-80.576820


In [3]:
spacex_df= spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launchsite_df = spacex_df.groupby(['Launch Site'],as_index=False).first()
launchsite_df = launchsite_df[['Launch Site', 'Lat', 'Long']]


In [4]:
launchsite_df.head()

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [5]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate,  zoom_start=10)

In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [7]:
site_map = folium.Map(location=nasa_coordinate,  zoom_start=5)

In [8]:
site = folium.map.FeatureGroup()

In [9]:
latitudes= list(launchsite_df['Lat'])
longitudes = list(launchsite_df['Long'])
name = list(launchsite_df['Launch Site'])

In [10]:
for lat ,long, name in zip(latitudes,longitudes,name):
    folium.Circle([lat,long],radius=1000, color='#d35400', fill=True).add_child(folium.Popup(name)).add_to(site_map)
    folium.map.Marker([lat,long],icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % name, )).add_to(site_map)
                  

In [11]:
site_map

In [12]:
launchsite_df.head()

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [13]:
marker_cluster = MarkerCluster()

In [14]:
# Apply a function to check the value of `class` column
def check(class_value):
    if class_value==1:
        return 'green'
    else:
        return 'red'
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(check)

In [15]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [16]:
site_map.add_child(marker_cluster)


In [17]:
# Create a MarkerCluster to hold the markers
marker_cluster = MarkerCluster().add_to(site_map)

# Iterate over each row in the spacex_df DataFrame
for index, record in spacex_df.iterrows():
    # Define the coordinates for each launch (assuming lat and long columns exist)
    coordinates = [record['Lat'], record['Long']]
    
    # Create a custom color for the marker based on the 'marker_color' column
    marker_color = record['marker_color']
    
    # Create the Marker object
    marker = folium.Marker(
        location=coordinates,
        icon=folium.Icon(color=marker_color, icon_color='white')
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the map
site_map


In [18]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [19]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [20]:
# find coordinate of the closet coastline
coastline_lat= 28.56321
coastline_lon= -80.56779
launch_site_lat=28.563197	
launch_site_lon=-80.576820
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [21]:
# Create a marker for the coastline point
distance_marker = folium.Marker([coastline_lat,coastline_lon],
                                icon=DivIcon(icon_size=(20, 20),
                                             icon_anchor=(0, 0),
                                             html='<div style="font-size:12px; color:#d35400;"><b> closest costline {:10.2f} KM</b></div>'.format(distance_coastline)
                                            )
                               )


# Add the distance marker to the map
distance_marker.add_to(site_map)

# Display the map
site_map


In [22]:
coastline_coordinate=[coastline_lat,coastline_lon]
launchsite_coordinate=[launch_site_lat,launch_site_lon]
coordinates = [coastline_coordinate, launchsite_coordinate]


In [23]:
# Create a PolyLine to connect the coastline and launch site
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue', opacity=0.7)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map

In [24]:
railway_cord = [28.5713, -80.5854]
highway_cord = [28.56281, -80.57073]
city_cord = [28.07834, -80.60774]
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_cord[0], railway_cord[1])
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_cord[0], highway_cord[1])
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_cord[0], city_cord[1])
target_cord = [railway_cord,highway_cord,city_cord]
distance_km = [distance_railway,distance_highway,distance_city]
name=['closest railway','closest highway','city']

In [25]:
for target, dis,name in zip(target_cord,distance_km,name):
    distance_marker = folium.Marker(target,
                                    icon=DivIcon(icon_size=(20, 20),
                                                 icon_anchor=(0, 0),
                                                 html='<div style="font-size:12px; color:#d35400;"><b> {} {:10.2f} KM</b></div>'.format(name,dis)
                                                )
                                   ).add_to(site_map)

    

# Display the map
site_map


In [26]:
for cord in target_cord:
    lines = folium.PolyLine(locations=[cord, launchsite_coordinate], weight=1, color='blue', opacity=0.7).add_to(site_map)
site_map